# Challenge 1

I attempted to use aprils data too to try to predict junes, however it considerably lowered the quality of the model: 

Loss: 0.0002993

MAE: 0.005401

RMSE: 0.01730

R²: 0.6074

Due to this, I am no longer using the models for it. However, may is an uncertain choice for predicting july's data. 

I also ruled out using relu, as it did not yield the results of tanh. 

In [16]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import Sequence 
from tensorflow.keras.optimizers import Adam     
 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,  mean_squared_log_error


In [17]:
df_omie_labelled = pd.read_csv('data\\df_omie_labelled.csv')
filtered_categories = pd.read_csv('data\\filtered_categories.csv')
df_omie_blind = pd.read_csv('data\\df_omie_blind.csv')
unit_list = pd.read_csv('data\\unit_list.csv')


unit_list["Tecnología"] = unit_list["Tecnología"].fillna("Unknown")

df_omie_labelled['fechaHora'] = pd.to_datetime(df_omie_labelled['fechaHora'])
df_omie_blind['fechaHora'] = pd.to_datetime(df_omie_blind['fechaHora'])

## Prepare Data

In [18]:
def prepare_data(df, train_months, test_month, columns_norm=["PrecEuro", "Energia"]):
    train_data = df[df['fechaHora'].dt.month.isin(train_months)]
    test_data = df[df['fechaHora'].dt.month == test_month]
     
     
    #fit scaler only on training data    
    scaler = MinMaxScaler()  
    train_data[columns_norm] = scaler.fit_transform(train_data[columns_norm])
    test_data[columns_norm] = scaler.transform(test_data[columns_norm])
    

    return train_data, test_data, scaler

In [19]:
class EnergySequence(Sequence): 
    def __init__(self, data, target_col, sequence_length=24, batch_size=32):
        self.data = data
        self.target_col = target_col
        self.sequence_length = sequence_length
        self.batch_size = batch_size
         

    def __len__(self):
        return (len(self.data) - self.sequence_length) // self.batch_size 
    

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = start_idx + self.batch_size
        X, y = [], []
   

        for i in range(start_idx, end_idx):
            if i + self.sequence_length < len(self.data):
                X.append(self.data.iloc[i : i + self.sequence_length].values)
                y.append(self.data.iloc[i + self.sequence_length, self.target_col])

        return np.array(X), np.array(y)   


##  Build Models 
(relu, tanh, combination)

In [20]:
def build_lstm_model(units, activations, input_shape):
    model = Sequential([
        LSTM(units[0], activation=activations[0], return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(units[1], activation=activations[1]),
        Dropout(0.2),
        Dense(units[2], activation=activations[2]),
        Dense(1)
    ])
    
    optimizer = Adam(learning_rate=0.001)  
    model.compile(optimizer=optimizer, loss='mse')
    return model


## Train & Evaluate Models

trains on april-may, may data to predict junes data. From here I will determine which model (am/m using relu/tanh/combination) is best to predict july

In [21]:
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(actual - predicted) / (np.abs(actual) + np.abs(predicted) + 1e-8))  

In [22]:
def train_evaluate_model(model, train_data, test_data, sequence_length, batch_size, epochs):
    
    train_generator = EnergySequence(train_data[['Energia', 'PrecEuro']], target_col=0, sequence_length=sequence_length, batch_size=batch_size)
    test_generator = EnergySequence(test_data[['Energia', 'PrecEuro']], target_col=0, sequence_length=sequence_length, batch_size=batch_size)

    history = model.fit(train_generator, validation_data=test_generator, epochs=epochs, verbose=1)

    loss = model.evaluate(test_generator)
    
    predictions = model.predict(test_generator)


    test_targets = test_data['Energia'].iloc[sequence_length:sequence_length+len(predictions)].values
    test_dates = test_data['fechaHora'].iloc[sequence_length:sequence_length+len(predictions)].values
    
    # error metrics
    mae = mean_absolute_error(test_targets, predictions)
    rmse = np.sqrt(mean_squared_error(test_targets, predictions))
    r2 = r2_score(test_targets, predictions)
    # mape = np.mean(np.abs((test_targets - predictions.flatten()) / test_targets)) * 100  
    mape = smape(test_targets, predictions.flatten())
    reg_accuracy = 100-mape

    return {
        "Loss": loss,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "MAPE": mape,
        "Regression Accuracy": reg_accuracy
    }


## Experiments - Predicint June

In [23]:
sequence_length = 24
batch_size = 64
epochs = 50  

 

train_may, test_june, scaler = prepare_data(df_omie_labelled, train_months=[5], test_month=6)  
# train_april_may, _, _ = prepare_data(df_omie_labelled, train_months=[4, 5], test_month=6)

# model_relu= build_lstm_model([30, 30, 20], ["relu", "relu", "relu"], (sequence_length, 2))
model_tanh= build_lstm_model([64, 64, 32], ["tanh", "tanh", "tanh"], (sequence_length, 2))
# model_relu_tanh = build_lstm_model([50, 50, 30], ["tanh", "tanh", "relu"], (sequence_length, 2))

# print("Training model (April-May --> June, ReLU)")  
# results_relu_am = train_evaluate_model(model_relu, train_april_may, test_june, sequence_length, batch_size, epochs)

# print("Training model (May --> June, ReLU)")   
# results_relu_m = train_evaluate_model(model_relu, train_may, test_june, sequence_length, batch_size, epochs)
  
# print("Training model (April-May --> June, Tanh)")
# results_tanh_am = train_evaluate_model(model_tanh, train_april_may, test_june, sequence_length, batch_size, epochs)

print("Training model (May --> June, Tanh)")
results_tanh_m = train_evaluate_model(model_tanh, train_may, test_june, sequence_length, batch_size, epochs)


# print("Training model (May --> June, Tanh/Relu)")
# results_relu_tanh_m = train_evaluate_model(model_relu_tanh, train_may, test_june, sequence_length, batch_size, epochs)


C:\Users\annap\AppData\Local\Temp\ipykernel_9928\4240801089.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[columns_norm] = scaler.fit_transform(train_data[columns_norm])
C:\Users\annap\AppData\Local\Temp\ipykernel_9928\4240801089.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[columns_norm] = scaler.transform(test_data[columns_norm])
c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape

Training model (May --> June, Tanh)
Epoch 1/50


c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12426/12426 ━━━━━━━━━━━━━━━━━━━━ 460s 36ms/step - loss: 3.4867e-04 - val_loss: 7.4617e-04
Epoch 2/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 538s 43ms/step - loss: 3.4926e-04 - val_loss: 7.2121e-04
Epoch 3/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 403s 32ms/step - loss: 3.2335e-04 - val_loss: 7.2255e-04
Epoch 4/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 479s 39ms/step - loss: 3.0777e-04 - val_loss: 6.6338e-04
Epoch 5/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 827s 67ms/step - loss: 2.7409e-04 - val_loss: 4.7246e-04
Epoch 6/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 3481s 280ms/step - loss: 2.7319e-04 - val_loss: 5.3459e-04
Epoch 7/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 853s 69ms/step - loss: 2.7574e-04 - val_loss: 4.0667e-04
Epoch 8/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 893s 72ms/step - loss: 2.4505e-04 - val_loss: 3.8235e-04
Epoch 9/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 871s 70ms/step - loss: 2.2063e-04 - val_loss: 2.9959e-04
Epoch 10/50
12426/12426 ━━━━━━━━━━━━━━━━━━━━ 889s 72ms/step - loss: 2.0336e-04 - val_loss: 3.2758e-0

In [24]:
# results_relu_am

In [25]:
# results_relu_m

In [26]:
# results_tanh_am

In [27]:
results_tanh_m

{'Loss': 0.00010760701115941629,
 'MAE': 0.004284586236129825,
 'RMSE': 0.010373377932773709,
 'R2': 0.8588608329819016,
 'MAPE': 133.4399663867288,
 'Regression Accuracy': -33.4399663867288}

## Results: 

results_tanh_m (epochs=20, sequence_length = 24, batch_size = 64, LSTM units=[50, 50, 30], learning rate=0.01): 

{'Loss': 0.0001265329192392528,

 'MAE': 0.004819060364125532,

 'RMSE': 0.011248683749875237,

 'R2': 0.8340372729665887,
 
 'MAPE': inf}